# Forecasting com a biblioteca Prophet
Artigo sobre o Prophet: https://peerj.com/preprints/3190/<br>
Documentação: https://facebook.github.io/prophet/docs/quick_start.html#python-api

In [2]:
%pip install pandas
%pip install gspread
%pip install gspread_dataframe
%pip install tqdm
%pip install prophet
%pip install python-dotenv
%pip install boto3
%pip install awswrangler

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to us

In [3]:
import os
import pandas as pd
import gspread
import gspread_dataframe as gd
import time
import awswrangler as wr
import boto3

from tqdm.auto import tqdm
from datetime import datetime
from prophet import Prophet
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
## Configurando credenciais google sheets
gc = gspread.service_account(filename = 'key.json')
sh = gc.open_by_key(os.environ.get('CODE_SHEET'))

## Extraindo os dados do Data Warehouse (DW)

In [8]:
def query_data_lake(query):
    my_session = boto3.session.Session(
        aws_access_key_id=os.environ.get('AWS_ACCESS_KEY_ID'),
        aws_secret_access_key=os.environ.get('AWS_SECRET_ACCESS_KEY'),
        region_name='us-east-1'
    )

    df = wr.athena.read_sql_query(query, database="dw_bs", ctas_approach=False, boto3_session=my_session)

    return df

In [15]:
def gerar_dados_completos():
    print('Generating data...')
    inicio = time.time()
    dados = query_data_lake(
    """
        select 
            date,
            business_model,
            sum(gmv) as gmv,
            sum(tickets) as ticket
        from ft_results
        where year(date) >= 2017
        group by 1, 2
        order by 1
    """
    )
    dados['gmv'] = dados['gmv'].astype('float')
    dados['ano'] = pd.DatetimeIndex(dados['date']).year
    dados['mes'] = pd.DatetimeIndex(dados['date']).month
    
    gd.set_with_dataframe(worksheet=sh.worksheet('Dados'), dataframe=dados, include_index=False, \
                          include_column_header=True, resize=True)
    
    total = dados.groupby('date', as_index=False)[['gmv', 'ticket']].sum()
    total.columns = ['date', 'gmv_total', 'ticket_total']

    pivot = pd.pivot_table(data=dados, index='date', columns=['business_model'], values=['gmv', 'ticket'])
    pivot = pivot.reset_index().dropna(axis=1)
    pivot.columns = ['date', 'gmv_ota', 'gmv_outras_otas', 'gmv_parc', 'gmv_wl', 'ticket_ota', 'ticket_outras_otas','ticket_parc', 'ticket_wl']
    pivot = pivot.drop(columns=['gmv_outras_otas','ticket_outras_otas'])
    
    join = pd.merge(pivot, total, on='date')
    join.to_csv('data/gmv_full.csv', sep=',', index=False)
    fim = time.time()
    
    print(f'Data generated! Time: {fim - inicio} s')

## Aplicando Prophet

In [10]:
def high_season(ds):
    date = pd.to_datetime(ds)
    return date.month == 12 or date.month == 7

In [11]:
def modelo(dados, metrica, data_corte, holidays):
    dados['ds'] = pd.to_datetime(dados['ds'])
    
    # Removendo dos dados o período da queda de vendas na pandemia
    dados.loc[(dados['ds'] > '2020-03-14') & (dados['ds'] < '2020-07-01'), 'y'] = None

    # Aplicando o regressor da high season
    dados['high_season'] = dados['ds'].apply(high_season)
    
    # Modelo preditivo 
    modelo = Prophet(holidays                = holidays,
                     daily_seasonality       = False, 
                     weekly_seasonality      = True, 
                     yearly_seasonality      = True,
                     interval_width          = 0.8,
                     n_changepoints          = 25,
                     changepoint_range       = 0.85,
                     changepoint_prior_scale = 0.05,
                     holidays_prior_scale    = 10.0,
                     seasonality_prior_scale = 5.0, 
                     seasonality_mode        = 'multiplicative')
    modelo.add_regressor('high_season')
    
    modelo.fit(dados[dados['ds'] <= data_corte])
    
    return modelo

In [12]:
def previsao(modelo, data_corte, data_previsao):
    # Definindo o período para previsão
    data_corte = datetime.strptime(data_corte, "%Y-%m-%d")
    data_previsao = datetime.strptime(data_previsao, "%Y-%m-%d")
    period = data_previsao - data_corte
    
    data_futuro = modelo.make_future_dataframe(periods=period.days)
    data_futuro['high_season'] = data_futuro['ds'].apply(high_season)
    previsao = modelo.predict(data_futuro)
    
#     previsao['yhat'] = previsao['yhat'] * 1.05 #apply(lambda x: x*1.05) 
    
    return previsao

In [13]:
# Variaveis a serem modificadas #
data_inicio_previsao = '2024-02-01'
data_fim_previsao = '2024-03-30'
gerar_dados = 1 # caso o dataset esteja desatualizado ou não exista, 1 para gerar ou atualizar e 0 caso contrário

In [16]:
metricas = ['gmv_total', 'gmv_ota', 'gmv_parc', 'gmv_wl', 'ticket_total', 'ticket_ota', 'ticket_parc', 'ticket_wl']

calendar = pd.read_table('data/calendar.tsv')
holidays = calendar[(calendar['ds'] >= '2017-01-01') & (calendar['ds'] <= '2023-12-31')]

if gerar_dados:
    gerar_dados_completos()

dataset = pd.read_csv('data/gmv_full.csv')
dt_cal = dataset['date'].iloc[-1]    # data até onde tem dados reais; ou pode ser definida uma outra data especifica

for metrica in tqdm(metricas):
    dados = dataset[['date', metrica]].copy()
    dados.columns = ['ds', 'y']
    
    model = modelo(dados, metrica, dt_cal, holidays) 
    forecast = previsao(model, dt_cal, data_fim_previsao)
    
    forecast = forecast[['ds', 'yhat_lower', 'yhat_upper', 'yhat']]
    forecast = forecast[forecast['ds'] >= data_inicio_previsao]
    
    dados['ds'] = pd.to_datetime(dados['ds']) # necessario para fazer o merge
    dados = dados[['ds', 'y']]
    
    forecast = pd.merge(forecast, dados, how='left', on='ds')
    
    gd.set_with_dataframe(worksheet=sh.worksheet(metrica), dataframe=forecast, include_index=False, \
                          include_column_header=True, resize=True)
    
#     forecast.to_csv('previsoes/'+metrica+'.csv', sep=',', index=False)

Generating data...
Data generated! Time: 13.658562898635864 s


  0%|          | 0/8 [00:00<?, ?it/s]/home/marlon/.local/lib/python3.8/site-packages/fbprophet/forecaster.py:171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.holidays['ds'] = pd.to_datetime(self.holidays['ds'])
/home/marlon/.local/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/home/marlon/.local/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/home/marlon/.local/lib/python3.

Initial log joint probability = -29.0459
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       6477.17    0.00766108       682.154      0.9204     0.09204      133   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       6557.57     0.0337314       860.428           1           1      251   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299        6634.6     0.0283515       1331.58           1           1      369   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399        6661.7     0.0177881       877.663           1           1      493   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       6674.48   0.000118589       356.763   3.201e-07       0.001      652  LS failed, Hessian reset 
    Iter      log prob        ||dx||    

/home/marlon/.local/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/home/marlon/.local/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/home/marlon/.local/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/home/marlon/.local/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.appen

Initial log joint probability = -20.9988
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       6564.41     0.0492292       1906.94           1           1      118   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       6671.63     0.0186937       293.119           1           1      240   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       6747.24     0.0025902       1091.92           1           1      357   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       6774.53     0.0015297       325.944           1           1      478   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       6820.36     0.0149057       315.206           1           1      601   
    Iter      log prob        ||dx||      ||grad||       alpha  

...

/home/marlon/.local/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/home/marlon/.local/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/home/marlon/.local/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/home/marlon/.local/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.appen

Initial log joint probability = -39.3658
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99        6100.1     0.0065671       586.753      0.1008           1      133   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       6142.55    0.00436073       432.036     0.09308           1      255   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       6171.61    0.00748455       206.467           1           1      380   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     346       6180.65   0.000120264       151.169   2.609e-07       0.001      475  LS failed, Hessian reset 
     399       6188.13     0.0190878       689.611           1           1      539   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       6199.55      0.065397    

/home/marlon/.local/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/home/marlon/.local/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/home/marlon/.local/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/home/marlon/.local/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.appen

Initial log joint probability = -76.3304
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       5776.98     0.0267072        1522.8           1           1      116   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       5863.34     0.0553639       1183.12      0.1078           1      236   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       5915.56    0.00611043       396.674      0.6081      0.6081      353   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399        5937.6     0.0294716       486.067           1           1      471   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       5956.31   0.000525887       372.048      0.6216      0.6216      584   
    Iter      log prob        ||dx||      ||grad||       alpha  

/home/marlon/.local/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/home/marlon/.local/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/home/marlon/.local/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/home/marlon/.local/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.appen